# DICOM - Draft

DICOM (Digital Imaging and Communications in Medicine) is a messaging standard used for data management of medical data and associated metadata.  It has the ability to integrate data from various medical imaging devices directly over TCP or UDP, or other more secure layers.  DICOM is a comprehensive specification of the data content, structure of the data, how the data is encoded, and relevant communication protocols.

Module learning objectives:
- DICOM data format 
- Displaying Images from the DICOM data format
- Manipulating and Normalizing Data

# 1. DICOM Data Format

The Digital Imaging and Communications in Medicine, the DICOM is a Standard originally published as NEMA standard for Digital Imaging and Communications in Medicine. The standard specifically deals with nonproprietary data interchange protocols, digital image formats, and file structure needed for biomedical images and image-related information sharing. 

Interesting blog about the DICOM standard can be found in this link: http://dicomiseasy.blogspot.com/2011/10/introduction-to-dicom-chapter-1.html

From programmer point of view, the core elements of dicom or in pydicom environment are
1. Dataset 
2. Data Element  
3. Tag
4. Sequence 

More resources on Dicom standard and description can be found in [4]

## DICOM data files

Some of the sample DICOM datasets are listed below 
* Osirix Datasets (http://www.osirix-viewer.com/resources/dicom-image-library/)   [Needs membership]
* Visible Human Datasets (https://mri.radiology.uiowa.edu/visible_human_datasets.html)   [Free Access to sample lib]
* Zubal Phantom (http://noodle.med.yale.edu/zubal/info.htm) [interesting]

## pydicom 

https://pypi.org/project/pydicom/
* ver: pydicom 1.2.0
* To Install: pip install pydicom
* very powerful tool in python to render DICOM images/files
* example file illustrated in pydicom

In [ ]:
! pip install --user pydicom 
! pip install --user opencv-python
! pip install --user scikit-image
! pip install --user ipywidgets

In [ ]:
# ! pip install dicom

## scikit-image 
For useful image-processing library and routines

In [ ]:
# reading a radiotherapy plan file
# import os
import numpy as np
import pydicom
import matplotlib.pyplot as plt
from pydicom.data import get_testdata_files
from skimage import data, io, filters

### Reading dicom file

* Similar to the Toy dataset, Pydicom has an inbuilt datasets.  we need to get_testdata_files() is used to load.
* .dcm file can be loaded using inbuilt reader - dcmread('filename')

In [ ]:
# using standard dataset
filename = get_testdata_files("rtplan.dcm")[0]
ds = pydicom.dcmread(filename)  # plan dataset

# Looking into the dicom file structure and type
print (type(ds), '\n\n')

# dicom data structure
print (ds)

### Reading .dcm file extensions (i.e., DICOM file/datasets) 

In [ ]:
ds = pydicom.dcmread("Data/vhf.1071.dcm")  # plan dataset
print(ds)

In [ ]:
ds.dir()

In [ ]:
print(ds.dir(), '\n')  

In [ ]:
ds.dir("Patient")

In [ ]:
ds.PatientPosition

In [ ]:
ds.PatientBirthTime

### Modifying individual attribute

In [ ]:
ds.PatientPosition = "HFP"
ds.PatientPosition

### Save .dcm file using save_as

In [ ]:
ds.save_as("MyCTScan.dcm")

## 2. Displaying Images from the DICOM data format

### __An example to view the test dataset in pydicom with matplot lib __ https://pydicom.github.io/pydicom/dev/viewing_images.html

### Dicom dataset file can be downloaded from https://mri.radiology.uiowa.edu/visible_human_datasets.html

###  Refer to the imshow in the dicom link provided above.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pydicom 
from pydicom.data import get_testdata_files

In [ ]:
filename = get_testdata_files("CT_small.dcm")[0]
ds_test = pydicom.dcmread(filename)

In [ ]:
print(ds_test)

In [ ]:
ds_test.dir('Patient')

In [ ]:
ds_test.PatientName

In [ ]:
ds_test.PatientSex

In [ ]:
ds_test.PatientAge

In [ ]:
ds_test.dir('Manufacturer')

In [ ]:
ds_test.ManufacturerModelName

## Working with Pixel data in dicom files

### Pixel array 
* We can display the dicom image using pixel_array
* In order to work on the image, the compressed and uncompressed data might need to be accessed and modified. Pixel_array provides a much easier access to the pixel data. 
* The modified data now needs to be saved, where in the previous function comes into picture/

### for imaging lets try with real .dcm file from reference [8]
#### vhf.1071.dcm is dicom file, female, head scan

In [ ]:
ds = pydicom.dcmread("Data/vhf.1071.dcm")

In [ ]:
ds.pixel_array

In [ ]:
ds.pixel_array.shape

In [ ]:
plt.figure(figsize = (8,8), dpi=100) 
plt.imshow(ds.pixel_array, cmap=plt.cm.gray) 

Note: cmap=plt.cm.gray provides a detailed and more serious look into the scan than without it as shown below.

In [ ]:
plt.figure(figsize = (8,8), dpi=100 )
plt.imshow(ds.pixel_array) 

### Pixel data 
Raw bytes of the file can be accessed using *PixelData*

In [ ]:
ds.PixelData 

In [ ]:
ds.pixel_array.flat

In [ ]:
plt.hist(ds.pixel_array.flatten(),bins=50)
plt.show()
print ("pixel array = ", ds.pixel_array.shape)
print("minimum value = ", np.amin(ds.pixel_array)) # find minimum pixel value in the image array
print("maximum value = ", np.amax(ds.pixel_array)) # find maximum pixel value in the image array

### Skimage 


### Rescaling and manipulation using skimage 
http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.histogram

In [ ]:
from skimage import exposure

plt.figure(figsize = (6,6), dpi=100)

p_lo, p_hi = np.percentile(ds.pixel_array, (40, 99))
img_rescale_1 = exposure.rescale_intensity(ds.pixel_array, in_range=(p_lo, p_hi))

figure = plt.imshow(img_rescale_1, cmap=plt.cm.gray)

In [ ]:
# There is a possibility to make interactive to handle the 
from ipywidgets import interactive, interact, widgets, Layout, Button, Box
from IPython.display import display
# turn off annoying pink warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def contrast_stretch(image_name, percentile_lo, percentile_hi):
    global img_rescale_interactive, image_name_global
    image_name_global = image_name 
    p_lo, p_hi = np.percentile(eval(image_name), (percentile_lo, percentile_hi))

    img_rescale = exposure.rescale_intensity(eval(image_name), in_range=(p_lo, p_hi))

    # save optimized image array to global variable so other functions can use it
    
    img_rescale_interactive = img_rescale
    
    plt.figure(figsize = (6,6), dpi=100) 
    plt.imshow(img_rescale, cmap=plt.cm.gray)
    
    plt.show()

In [ ]:
w = interactive(contrast_stretch, image_name="ds.pixel_array", percentile_lo=(1,100,.5), percentile_hi=(1,100,.5))

w

### Flipping images 
- Left to Right

In [ ]:
plt.figure(figsize = (6,6), dpi=100)
plt.imshow(np.fliplr(ds.pixel_array), cmap=plt.cm.gray)

* Upside down

In [ ]:
plt.figure(figsize = (6,6), dpi=100)
plt.imshow(np.flipud(ds.pixel_array), cmap=plt.cm.gray)

Other medical Imaging python libraries
* __MedPy__
* __scikit-image__

Note: DICOM standard has gained significant importance with technology and currently there are various DICOM viewer available for public to use. The Android and APP store current hold quite a few viewer.   

Online Dicom file viwer is shown below
https://www.dicomlibrary.com/meddream/md5/index.html?study=1.2.826.0.1.3680043.8.1055.1.20111103111148288.98361414.79379639

## Excercise


###  <font color='red'> Downloaded dicom file from the link is given in your labs/Data folder. Follow each task below. </font>

### https://mri.radiology.uiowa.edu/visible_human_datasets.html
 
 * Load two dicom files - vhf.1608.dcm & image-000000.dcm  
 * Assign to ds5 and ds6
 * investigate the pixel information
 * display scan using imshow from matplotlib 

In [ ]:
# read file vhf.1608.dcm
# hint use: dcmread

ds5 = 
print ('ds5 info', .shape) # print shape

# knee image-000000.dcm
ds6 = 
print ('ds6 info', )

In [ ]:
ds5.pixel_array

###  <font color='red'>  Viewing Images from dicom </font>

https://pydicom.github.io/pydicom/stable/viewing_images.html

In [ ]:
#cmap = plt.cm.gray


###  <font color='red'> Does this dicom file display any of the Patient's information?</font>


In [ ]:
ds5

### If patient name is shared then remove the records or replace with dummy name

In [ ]:
ds5.

In [ ]:
ds5. = # say 'dummy01'

In [ ]:
ds5.

#### Note: The operation we perform to hide the patient sensitive information is simple words known as Anonymized. 


###  <font color='red'> Use the *contrast_stretch()* function shown above and explore the parameters to get the decent image</font>


In [ ]:
interactive()

display()

In [ ]:
interactive()

display()

###  <font color='red'> Demonstrate the flip function for above </font>

###  <font color='red'>Save to jpeg or png or tif </font>

### Use this link for help https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imsave.html



# References & further reading
1. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC61235/
2. http://uwmsk.org/jupyter/Jupyter_DICOM_toolbox.html
3. https://medium.com/@taposhdr/medical-image-analysis-with-deep-learning-i-23d518abf531
4. https://www.dicomstandard.org/
5. https://pyscience.wordpress.com/2014/09/08/dicom-in-python-importing-medical-image-data-into-numpy-with-pydicom-and-vtk/
6. Oleg S. Pianykh, "Digital Imaging and Communications in Medicine (DICOM): A Practical Introduction and Survival Guide", Springer 2012. (available to MTU students:: https://link.springer.com/book/10.1007%2F978-3-642-10850-1)
7. Oleg S. Pianykh, "Digital Image Quality in Medicine", Springer 2014 (available to MTU students:: https://link.springer.com/book/10.1007%2F978-3-319-01760-0)
8. https://mri.radiology.uiowa.edu/visible_human_datasets.html

Other helpful Resources:
- https://www.dicomlibrary.com/
- http://dicomiseasy.blogspot.com/

